# Culinária ao Redor do Mundo: Análise de Receitas

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [ ]:
%defaultDatasource jdbc:h2:mem:db

Criação de Tabela

In [ ]:
DROP TABLE IF EXISTS Receitas;

CREATE TABLE Receitas (
  id INTEGER NOT NULL,
  titulo VARCHAR(200),
  regiao VARCHAR(100),
  PRIMARY KEY(id)
) AS SELECT
  id, titulo, regiao
FROM CSVREAD('../data/processed/receitas.csv');


In [ ]:
DROP TABLE IF EXISTS Nutrientes;

CREATE TABLE Nutrientes (
    id INTEGER NOT NULL,
    nome VARCHAR(50),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/nutrientes.csv');

In [ ]:
DROP TABLE IF EXISTS Compostos;

CREATE TABLE Compostos (
    id INTEGER NOT NULL,
    nome VARCHAR(300),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/compostos.csv');

In [ ]:
DROP TABLE IF EXISTS Ingredientes;

CREATE TABLE Ingredientes (
    id_cdb INTEGER NOT NULL,
    nome_cdb VARCHAR(300),
    id_fdb INTEGER NOT NULL,
    nome_fdb VARCHAR(300),
    grupo VARCHAR(300),
    subgrupo VARCHAR(300),
    aliases VARCHAR(300),
    PRIMARY KEY(id_cdb, id_fdb)
) AS SELECT
    id_cdb, nome_cdb, id_fdb, nome_fdb, grupo, subgrupo, aliases
FROM CSVREAD('../data/interim/ingredientes_final.csv');

In [ ]:
DROP TABLE IF EXISTS ingredientes_receitas;

CREATE TABLE Ingredientes_receitas (
    id_ingrediente INTEGER NOT NULL,
    id_receita INTEGER NOT NULL,
    volume FLOAT NOT NULL,
    massa FLOAT NOT NULL,
    unidade INTEGER NOT NULL,
    PRIMARY KEY (id_ingrediente, id_receita)
) AS SELECT
    id_ingrediente, id_receita, volume, massa, unidade
FROM CSVREAD('../data/processed/ingredientes_receitas.csv');

In [ ]:
DROP TABLE IF EXISTS Compostos_content;

CREATE TABLE Compostos_content (
  id_ingrediente INTEGER NOT NULL,
  id_composto INTEGER NOT NULL,
  quantidade FLOAT NOT NULL,
  PRIMARY KEY (id_ingrediente, id_composto)
) AS SELECT
    id_ingrediente, id_composto, quantidade
FROM CSVREAD('../data/processed/content_compostos.csv')

In [ ]:
DROP TABLE IF EXISTS Nutrientes_content;

CREATE TABLE Nutrientes_content (
  id_ingrediente INTEGER NOT NULL,
  id_nutriente INTEGER NOT NULL,
  quantidade FLOAT NOT NULL,
  PRIMARY KEY (id_ingrediente, id_nutriente)
) AS SELECT
    id_ingrediente, id_nutriente, quantidade
FROM CSVREAD('../data/processed/content_nutrientes.csv')

1)Quais as estruturas bioquímicas mais frequentes em cada região?

In [ ]:
DROP VIEW IF EXISTS MostCommonCompoundByRegion;
CREATE VIEW MostCommonCompoundByRegion AS
SELECT
    r.regiao AS nome_regiao,
    r.titulo AS nome_receita,
    c.nome AS nome_composto,
    subq.quantidade
FROM
    Receitas r
JOIN
    (
        SELECT
            r1.regiao AS regiao_subq,
            r1.titulo AS titulo_subq,
            c1.nome AS composto_subq,
            COUNT(*) AS quantidade
        FROM
            Receitas r1
        JOIN
            Ingredientes_receitas ir1 ON r1.id = ir1.id_receita
        JOIN
            Ingredientes i1 ON ir1.id_ingrediente = i1.id_cdb
        JOIN
            Compostos_content cc1 ON i1.id_cdb = cc1.id_ingrediente
        JOIN
            Compostos c1 ON cc1.id_composto = c1.id
        GROUP BY
            r1.regiao, r1.titulo, c1.nome
    ) subq
ON
    r.regiao = subq.regiao_subq AND r.titulo = subq.titulo_subq
JOIN
    Compostos c ON subq.composto_subq = c.nome;
SELECT * FROM MostCommonCompoundByRegion LIMIT 20;

2)Quais regiões possuem receitas com ingredientes majoritariamente vegetais?

In [ ]:
DROP VIEW IF EXISTS RegionVegetablePercentageRanking;
CREATE VIEW RegionVegetablePercentageRanking AS
SELECT
    r.regiao AS nome_regiao,
    COUNT(DISTINCT CASE WHEN i.grupo NOT IN ('Milk and milk products', 'Snack foods', 'Aquatic foods', 'Animal foods') THEN ir.id_ingrediente END) AS total_vegetais,
    COUNT(DISTINCT ir.id_ingrediente) AS total_ingredientes,
    COUNT(DISTINCT ir.id_receita) AS total_receitas,
    (COUNT(DISTINCT CASE WHEN i.grupo NOT IN ('Milk and milk products', 'Snack foods', 'Aquatic foods', 'Animal foods') THEN ir.id_ingrediente END) * 100.0) / COUNT(DISTINCT ir.id_ingrediente) AS porcentagem_vegetais
FROM
    Receitas r
JOIN
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN
    Ingredientes i ON ir.id_ingrediente = i.id_cdb
GROUP BY
    r.regiao
ORDER BY
    porcentagem_vegetais;
SELECT * FROM RegionVegetablePercentageRanking LIMIT 20;


3)Quais regiões possuem a maior média de gorduras por receita?

In [ ]:
DROP VIEW IF EXISTS RegionFatAverageRanking;
CREATE VIEW RegionFatAverageRanking AS
SELECT
    r.regiao AS nome_regiao,
    AVG(nc.quantidade) AS media_gordura
FROM
    Receitas r
JOIN
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN
    Nutrientes_content nc ON ir.id_ingrediente = nc.id_ingrediente
WHERE
    nc.id_nutriente = 1
GROUP BY
    r.regiao
ORDER BY media_gordura;
SELECT * FROM RegionFatAverageRanking LIMIT 20;

org.h2.jdbc.JdbcSQLException:  Syntax error in SQL statement "[*]/*NÃO DÁ/*"; SQL statement

4) Qual a porcentagem de receitas que usam frutas tropicais por região?

In [ ]:
DROP VIEW IF EXISTS RegionTropicalFruitsPercentage
CREATE VIEW RegionTropicalFruitsPercentage AS
SELECT
    r.regiao AS nome_regiao,
    COUNT(DISTINCT CASE WHEN i.subgrupo = 'Tropical fruits' THEN ir.id_receita END) * 100.0 / COUNT(DISTINCT ir.id_receita) AS porcentagem_tropical_fruits
FROM
    Receitas r
JOIN
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN
    Ingredientes i ON ir.id_ingrediente = i.id_cdb
GROUP BY
    r.regiao;
SELECT * FROM RegionTropicalFruitsPercentage LIMIT 20;